In [22]:
import pandas as pd
import numpy as np

In [23]:
# pd.set_option('display.max_rows', None)
pd.reset_option('display.max_rows')

In [26]:
class cleanData:
    def __init__(self, raw_data, question_dictionary, roster, normalization=True, mapping=True):
        self.raw_data = raw_data
        self.question_dict = question_dictionary
        self.roster = roster
        
        self.subset_data = raw_data
        self.needsNormalization = normalization
        self.needsMapping = mapping
    
    # assert all files are of .csv extension 
    def assertCSVFiles(self):
        if not "CSV" in self.raw_data.upper():
            raise Exception("raw data file is not in CSV format")
        if not "CSV" in self.roster.upper():
            raise Exception("roster file is not in CSV format")
        if not "CSV" in self.question_dict.upper():
            raise Exception("question dictionary file is not in CSV format")
        
        # if still want to check all files together
        # if not any(["CSV" in [self.raw_data.upper(), self.roster.upper(), self.question_dict.upper()]]):
        #     raise Exception("At least one of the input files is not in CSV format")

    # read all CSV files 
    def readCSVs(self):
        self.raw_data = pd.read_csv(self.raw_data)
        self.question_dict = pd.read_csv(self.question_dict)
        self.roster = pd.read_csv(self.roster)

    # return the list of quantitative questions from question_dictionary
    def getQuantQuestions(self):
        quant_dict = self.question_dict[self.question_dict["type"] == "quantitative"]
        return list(quant_dict["question_id"])

    # subset raw data to just student email, student first name, student last name, and all question responses 
    def getSubset(self):
        cols_needed = []
        keywords_list = ['EMAIL', 'FIRST', 'LAST', 'Q']
        for col in list(self.raw_data.columns):
            if any(keyword in col.upper() for keyword in keywords_list):
                cols_needed.append(col)

        # update class variable subset_data to the correct subset
        self.subset_data = self.raw_data[cols_needed]

        # replace question column names in subset data with X.Y instead of QX_Y
        self.subset_data.columns = [col.replace('Q', '').replace('_', '.') for col in list(self.subset_data.columns)]
        return self.subset_data

    # join in Team Number and TeammateNumber from roster; drop rows of metadata
    def joinRosterAndRaw(self):
        roster_email_field = [col for col in list(self.roster.columns) if 'EMAIL' in col.upper()][0]
        cleaned_email_field = [col for col in list(self.getSubset().columns) if 'EMAIL' in col.upper()][0]

        # joins roster (Email) and raw (RecipientEmail)'s subset data on same email address
        full_cleaned = pd.merge(self.roster[[roster_email_field, 'Team Number', 'TeammateNumber']], 
                                self.subset_data, 
                                how="outer", 
                                left_on = roster_email_field, 
                                right_on = cleaned_email_field)
        
        # drop N/A rows, and sort df by Team Number then TeammateNumber starting from Team1
        full_cleaned = full_cleaned[~full_cleaned["Team Number"].isna()]
        full_cleaned = full_cleaned.sort_values(["Team Number", "TeammateNumber"]).reset_index().drop("index", axis=1)

        # removes redundancy of the 2 emails columns 
        full_cleaned = full_cleaned.drop('RecipientEmail', axis=1)
        return full_cleaned
        
    # fix numbers in quantitative questions that are identified as strings into integers
    def stringToNumerical(self, data):
        for question in self.getQuantQuestions():
            question_str = str(question)
            lst = data[question_str].str.contains(r'\d+')
            filtered_lst = [x for x in lst if not (x is np.nan)]
            if any(filtered_lst):
                data[question_str] = pd.to_numeric(data[question_str])   
        return data

    def sortColumns(self, data):
        # transform data columns into float for sorting purpose
        for column in data.columns[5:]:  # 6 can change based on joinRosterAndRaw()
            data.rename(columns={column: float(column)}, inplace=True)

        sorted = data.iloc[:, 5:]
        sorted = sorted.sort_index(axis=1)
        sorted['Email'] = data['Email']
        data = data.iloc[:, :5]    
        
        return pd.merge(data, sorted, on='Email')
        
    def mappingScale(self, maxScale):
        seven_scale_mappings = {
        "STRONGLY AGREE" : 7,
        "AGREE" : 6,
        "SOMEWHAT AGREE" : 5,
        "NEITHER AGREE NOR DISAGREE" : 4,
        "SOMEWHAT DISAGREE" : 3,
        "DISAGREE" : 2,
        "STRONGLY DISAGREE" : 1,
        np.nan : np.nan,
        "MUCH BETTER" : 7,
        "MODERATELY BETTER" : 6,
        "SLIGHTLY BETTER" : 5,
        "ABOUT THE SAME" : 4,
        "SLIGHTLY WORSE" : 3,
        "MODERATELY WORSE" : 2,
        "MUCH WORSE" : 1,
        }

        five_scale_mappings = {
            "ALWAYS" : 5,
            "VERY OFTEN" : 4,
            "SOMETIMES" : 3,
            "RARELY" : 2,
            "NEVER" : 1,
            np.nan : np.nan
        }
        if maxScale == 7:
            return seven_scale_mappings
        return five_scale_mappings

    # if the raw data has "Agree"/"Disagree" in quantitative question columns, map these to integers 1-X where X is "out_of"
    def mapData(self, data):
        for question in self.getQuantQuestions():
            # get the max scale of current question
            maxScale = int(self.question_dict[self.question_dict["question_id"] == question]["out_of"])
            scaleFunction = self.mappingScale(maxScale)
            question_str = str(question)
            
            if maxScale == 7 or maxScale == 5:
                data[question_str] = data[question_str].str.upper()
    
                # starts mapping relevant questions
                lstOfResponses = []
                for response in data[question_str]:
                    lstOfResponses.append(scaleFunction[response])
                data[question_str] = lstOfResponses
        return data
        
    # if normalization is true, then all quantitative data is normed to 0
    # assumed question_dictionary includes "out_of" column
    def normalize(self, data):
        for question in self.getQuantQuestions():
            # get the max scale of current question
            maxScale = int(self.question_dict[self.question_dict["question_id"] == question]["out_of"])
            scaleFunction = self.mappingScale(maxScale)
            question_str = str(question)
            
            # if series values are agree/disagree, adjust the scale for visualization purpose
            if maxScale == 7:
                data[question_str] = pd.to_numeric(data[question_str]) - 4
            else:
                data[question_str] = pd.to_numeric(data[question_str]) - 3
        return data

    # For NA values (students that didn't complete survey, left question empty), fill in with "No Response"
    def fillNA(self, data):
        data = data.fillna('No Response')
        return data

    def main(self):
        self.assertCSVFiles()
        # reads CSVs
        self.readCSVs()

        # joins subset of roster and raw data
        data = self.joinRosterAndRaw()
        print(data.columns)
        data = self.stringToNumerical(data)
        
        if self.needsMapping:
            data = self.mapData(data)

        if self.needsNormalization:
            data = self.normalize(data)
        data = self.sortColumns(data)
        data = self.fillNA(data)
        return data